# Analysis and Visualization

Let's create a visualization of a single meme:

## 0. Setup

In [2]:
import os
import os.path

from kgtk.configure_kgtk_notebooks import ConfigureKGTK
from kgtk.functions import kgtk, kypher

In [3]:
# Parameters

# Folders on local machine where to create the output and temporary files:
input_path = "wikidata"
output_path = "projects"
project_name = "tutorial-kypher"

In [4]:
big_files=["label"]

additional_files = {
    "P31": "derived.P31.tsv.gz",
    "items": "claims.wikibase-item.tsv.gz",
    "P1963": "derived.P1963computed.count.star.tsv.gz",
    "external": "claims.external-id.tsv.gz",
    "indegree": "metadata.in_degree.tsv.gz",
    "outdegree": "metadata.out_degree.tsv.gz",
    "pagerank": "metadata.pagerank.directed.tsv.gz"
}

ck = ConfigureKGTK(big_files)
ck.configure_kgtk(input_graph_path=input_path, 
                  output_path=output_path, 
                  project_name=project_name,
                  additional_files=additional_files)

User home: /Users/filipilievski
Current dir: /Users/filipilievski/mcs/imkg
KGTK dir: /Users/filipilievski/mcs
Use-cases dir: /Users/filipilievski/mcs/use-cases


In [5]:
ck.print_env_variables()

kypher: kgtk query --graph-cache projects/tutorial-kypher/temp.tutorial-kypher/wikidata.sqlite3.db
GRAPH: wikidata
KGTK_LABEL_FILE: wikidata/labels.en.tsv.gz
USE_CASES_DIR: /Users/filipilievski/mcs/use-cases
OUT: projects/tutorial-kypher
KGTK_OPTION_DEBUG: false
STORE: projects/tutorial-kypher/temp.tutorial-kypher/wikidata.sqlite3.db
EXAMPLES_DIR: /Users/filipilievski/mcs/examples
kgtk: kgtk
KGTK_GRAPH_CACHE: projects/tutorial-kypher/temp.tutorial-kypher/wikidata.sqlite3.db
TEMP: projects/tutorial-kypher/temp.tutorial-kypher
label: wikidata/labels.en.tsv.gz
P31: wikidata/derived.P31.tsv.gz
items: wikidata/claims.wikibase-item.tsv.gz
P1963: wikidata/derived.P1963computed.count.star.tsv.gz
external: wikidata/claims.external-id.tsv.gz
indegree: wikidata/metadata.in_degree.tsv.gz
outdegree: wikidata/metadata.out_degree.tsv.gz
pagerank: wikidata/metadata.pagerank.directed.tsv.gz


Create label file:

In [ ]:
!kgtk query -i $label -i $TEMP/templates.kgtk.gz \
            --match 'templates: ()-[]->(n), \
                label: (n)-[r]->(l)' \
            --return 'n as node1, r.label as label, l as node2' / deduplicate \
            -o $TEMP/labelfile.kgtk.gz 

## 1. Visualize a single meme

In [15]:
!kgtk query -i $TEMP/templates.kgtk.gz \
            --match '(:`kym:distracted-boyfriend`)-[r]->()' \
             --where 'r.label in ["kym:parent", "kym:child", "kym:year", "rdf:type", "m4s:fromAbout", "m4s:fromTags", "m4s:fromImage"]' \
            -o $TEMP/db_subject.kgtk.gz

node1	label	node2
kym:distracted-boyfriend	dbp:confidence	0.5
kym:distracted-boyfriend	kym:about	Distracted Boyfriend, also known as Man Looking at Other Woman, is an object labeling stock photo series in which a man looks at the backside of a woman walking by while another woman, presumably his romantic partner, looks on disapprovingly.
kym:distracted-boyfriend	kym:added	nodeHtb5rZc6kCD2VNGVWKVeG4-669
kym:distracted-boyfriend	kym:from	iStock
kym:distracted-boyfriend	kym:last_update_source	nodeHtb5rZc6kCD2VNGVWKVeG4-670
kym:distracted-boyfriend	kym:origin	On November 2, 2015, the source image, taken by photographer Antonio Guillem, was posted on the stock photo database iStock[1] under the description \Disloyal man with his girlfriend looking at another girl\" (shown below, left). According to a post published on the Meme Documentation [14] Tumblr blog, the earliest known captioned version of the photo was submitted to a Turkish Facebook group at an unknown date. On January 30th, 2017,

In [17]:
!kgtk query -i $TEMP/templates.kgtk.gz \
            --match '(:`kym:distracted-boyfriend`)-[r]->()' \
            --where 'r.label in ["kym:parent", "kym:sibling"]' \
            -o $TEMP/db_subject.kgtk.gz

Create node file:

In [18]:
!kgtk query -i $TEMP/labelfile.kgtk.gz -i $TEMP/db_subject.kgtk.gz \
            --match 'db: ()-[]->(n), \
                label: (n)-[r]->(l)' \
            --return 'n as id, l as label' / deduplicate \
            -o $TEMP/nodefile.kgtk.gz 

In [19]:
kgtk('''
    cat -i $TEMP/nodefile.kgtk.gz
''')

,id,label


In [20]:
kgtk('''
    cat -i $TEMP/db_subject.kgtk.gz
''')

,node1,label,node2
0,kym:distracted-boyfriend,kym:parent,kym:object-labeling
1,kym:distracted-boyfriend,kym:sibling,kym:a-good-book-can-change-your-life
2,kym:distracted-boyfriend,kym:sibling,kym:another-one-bites-the-dust
3,kym:distracted-boyfriend,kym:sibling,kym:assassination-chain
4,kym:distracted-boyfriend,kym:sibling,kym:bangladeshi-cake-cutting
5,kym:distracted-boyfriend,kym:sibling,kym:bending-streetlight
6,kym:distracted-boyfriend,kym:sibling,kym:big-guy-ordering-at-mcdonalds
7,kym:distracted-boyfriend,kym:sibling,kym:boat-pulling-cruise-ship-through-corinth-c...
8,kym:distracted-boyfriend,kym:sibling,kym:bronze-medal
9,kym:distracted-boyfriend,kym:sibling,kym:butt-poke


In [21]:
kgtk("""visualize-graph 
        -i $TEMP/db_subject.kgtk.gz
        --node-file $TEMP/nodefile.kgtk.gz
        --show-text above
        --tooltip-column label
        --direction arrow
        --edge-color-column label
        --edge-color-style d3.schemeDark2
        -o distracted.graph.html""")

In [27]:
!grep "kym:" imkg02/instances.imgflip.nt | head

Issue: how are the instances and the templates linked?